In [1]:
# 프로그램 5-1: 해리스 특징점 검출 구현하기

import cv2 as cv          # OpenCV를 cv로 import
import numpy as np        # numpy를 np로 import

# 입력 영상: 10x10 크기의 2진 영상 (코너 모양을 갖는 형태)
img = np.array([[0,0,0,0,0,0,0,0,0,0],
                [0,0,0,0,0,0,0,0,0,0],
                [0,0,1,1,0,0,0,0,0,0],
                [0,0,1,1,1,0,0,0,0,0],
                [0,0,1,1,1,1,0,0,0,0],
                [0,0,1,1,1,1,1,0,0,0],
                [0,0,1,1,1,1,1,1,0,0],
                [0,0,0,0,0,0,0,0,0,0],
                [0,0,0,0,0,0,0,0,0,0],
                [0,0,0,0,0,0,0,0,0,0]], dtype=np.float32)

# x방향, y방향 미분을 위한 커널 정의
ux = np.array([[-1, 0, 1]])               # x 방향 커널
uy = np.array([-1, 0, 1]).transpose()     # y 방향 커널 (세로로 변환)

# 1차원 가우시안 커널 생성 (크기: 3, 표준편차 자동)
k = cv.getGaussianKernel(3, 1)

# 2차원 가우시안 커널 생성 (외적 사용)
g = np.outer(k, k.transpose())

# 영상의 y방향, x방향 미분 계산
dy = cv.filter2D(img, cv.CV_32F, uy)      # y 방향 미분
dx = cv.filter2D(img, cv.CV_32F, ux)      # x 방향 미분

# 해리스 반응을 위한 항들 계산
dyy = dy * dy                             # Iy^2
dxx = dx * dx                             # Ix^2
dyx = dy * dx                             # Ix * Iy

# 위 항들에 가우시안 블러 적용
gdyy = cv.filter2D(dyy, cv.CV_32F, g)
gdxx = cv.filter2D(dxx, cv.CV_32F, g)
gdyx = cv.filter2D(dyx, cv.CV_32F, g)

# 해리스 코너 응답 함수 계산
# C = det(M) - k * trace(M)^2
# det = gdyy * gdxx - gdyx^2, trace = gdyy + gdxx
C = (gdyy * gdxx - gdyx * gdyx) - 0.04 * (gdyy + gdxx)**2

#비최대 억제
# 코너 응답 결과에서 실제 코너를 탐색
for j in range(1, C.shape[0]-1):                  # 세로 방향
    for i in range(1, C.shape[1]-1):              # 가로 방향
        # 임계값을 넘고 주변보다 클 경우
        if C[j, i] > 0.1 and sum(sum(C[j-1:j+2, i-1:i+2] > C[j, i])) == 8:
            img[j, i] = 9                         # 특징점을 원본 영상에 9로 표시

# 출력 형식 설정 (소수점 2자리까지)
np.set_printoptions(precision=2)

# 중간 결과 출력 (디버깅용)
print(dy)        # ① y 방향 미분
print(dx)        # ② x 방향 미분
print(dyy)       # ③ dy^2
print(dxx)       # ④ dx^2
print(dyx)       # ⑤ dx*dy
print(gdyy)      # ⑥ dy^2에 가우시안 적용
print(gdxx)      # ⑦ dx^2에 가우시안 적용
print(gdyx)      # ⑧ dx*dy에 가우시안 적용
print(C)         # ⑨ 해리스 응답값                 #특징 가능성 맵
print(img)       # ⑩ 코너 표시된 영상              #특징점을 9로 표시한 원본 영상

# 시각화를 위한 160x160 확대 배열 생성
popping = np.zeros([160, 160], np.uint8)

# 10x10 결과를 16배 확대 (16x 확대 = 160x160)
for j in range(0, 160):             
    for i in range(0, 160):
        popping[j, i] = np.uint8((C[j//16, i//16] + 0.06) * 700)  # 대비 조정

# 영상 출력
cv.imshow('Image Display2', popping)  # ⑪ 응답 맵 시각화
cv.waitKey()
cv.destroyAllWindows()

[[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  1.  1.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  1.  1.  1.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  1.  1.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  1.  1.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  1.  1.  0.  0.]
 [ 0.  0. -1. -1. -1. -1. -1.  0.  0.  0.]
 [ 0.  0. -1. -1. -1. -1. -1. -1.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]]
[[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  1.  1. -1. -1.  0.  0.  0.  0.  0.]
 [ 0.  1.  1.  0. -1. -1.  0.  0.  0.  0.]
 [ 0.  1.  1.  0.  0. -1. -1.  0.  0.  0.]
 [ 0.  1.  1.  0.  0.  0. -1. -1.  0.  0.]
 [ 0.  1.  1.  0.  0.  0.  0. -1. -1.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]]
[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 1. 1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 1. 1. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 1. 0. 0. 0. 0.]
 [0. 0

In [5]:
#프로그램 5-2 SIFT 검출
import cv2 as cv

img = cv.imread('mot_color70.jpg')  #영상 읽기
gray = cv.cvtColor(img, cv.COLOR_RGB2GRAY)

sift= cv.SIFT_create()
kp,des=sift.detectAndCompute(gray,None)

gray=cv.drawKeypoints(gray, kp, None, flags=cv.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)
cv.imshow('sift', gray)

k=cv.waitKey()
cv.destroyAllWindows()

In [6]:
# 프로그램 5-3 FLANN 라이브러리를 이용한 SIFT 매칭

import cv2 as cv
import numpy as np
import time

img1 = cv.imread('mot_color70.jpg')[190:350, 440:560]  # 버스를 크롭하여 모델 영상으로 사용
gray1 = cv.cvtColor(img1, cv.COLOR_BGR2GRAY)
img2 = cv.imread('mot_color83.jpg')                    # 장면 영상
gray2 = cv.cvtColor(img2, cv.COLOR_BGR2GRAY)

sift = cv.SIFT_create()
kp1, des1 = sift.detectAndCompute(gray1, None)
kp2, des2 = sift.detectAndCompute(gray2, None)
print('특징점 개수:', len(kp1), len(kp2))  # ①

start = time.time()
flann_matcher = cv.DescriptorMatcher_create(cv.DescriptorMatcher_FLANNBASED)
knn_match = flann_matcher.knnMatch(des1, des2, 2)

T = 0.7
good_match = []
for nearest1, nearest2 in knn_match:
    if (nearest1.distance / nearest2.distance) < T:  # 식 (5.14)의 최근접 이웃 거리 비율 적용
        good_match.append(nearest1)
print('매칭에 걸린 시간:', time.time() - start)  # ②

img_match = np.empty((max(img1.shape[0], img2.shape[0]), img1.shape[1] + img2.shape[1], 3), dtype=np.uint8)
cv.drawMatches(img1, kp1, img2, kp2, good_match, img_match, flags=cv.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS)

cv.imshow('Good Matches', img_match)

k = cv.waitKey()
cv.destroyAllWindows()


특징점 개수: 231 4096
매칭에 걸린 시간: 0.05510997772216797


In [11]:
# 프로그램 5-4 RANSAC을 이용해 호모그래피 추정하기

import cv2 as cv
import numpy as np

img1 = cv.imread('mot_color70.jpg')[190:350, 440:560]  # 버스를 크롭하여 모델 영상으로 사용
gray1 = cv.cvtColor(img1, cv.COLOR_BGR2GRAY)
img2 = cv.imread('mot_color83.jpg')                    # 장면 영상
gray2 = cv.cvtColor(img2, cv.COLOR_BGR2GRAY)

sift = cv.SIFT_create()
kp1, des1 = sift.detectAndCompute(gray1, None)
kp2, des2 = sift.detectAndCompute(gray2, None)

flann_matcher = cv.DescriptorMatcher_create(cv.DescriptorMatcher_FLANNBASED)
knn_match = flann_matcher.knnMatch(des1, des2, 2)      # 최근접 2개

T = 0.7
good_match = []
for nearest1, nearest2 in knn_match:
    if (nearest1.distance / nearest2.distance) < T:
        good_match.append(nearest1)

points1 = np.float32([kp1[gm.queryIdx].pt for gm in good_match])
points2 = np.float32([kp2[gm.trainIdx].pt for gm in good_match])

H,_=cv.findHomography(points1, points2, cv.RANSAC)

h1, w1 = img1.shape[0], img1.shape[1]  #1번 영상 크기
h2, w1 = img2.shape[0], img2.shape[1]   # 2번 영상 크기

box1=np.float32([[0,0], [0,h1-1], [w1-1, h1-1], [w1-1,0]]).reshape(4,1,2)
cv.drawMatches(img1, kp1, img2, kp2, good_match,img_match,flags=cv.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS)

cv.imshow('Matches and Homography', img_match)

k=cv.waitKey()
cv.destroyAllWindows()
